In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import silhouette_score
from prophet import Prophet
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# 1. Data Loading and Initial Preprocessing
def load_and_preprocess_data(file_path):
    """
    Load and preprocess the retail dataset
    """
    # Load data
    df = pd.read_csv(file_path)

    # Convert Order Date to datetime
    df['Order Date'] = pd.to_datetime(df['Order Date'], format='%d-%m-%Y')

    # Create time-based features
    df['Year'] = df['Order Date'].dt.year
    df['Month'] = df['Order Date'].dt.month
    df['Day of Week'] = df['Order Date'].dt.dayofweek
    df['Is Weekend'] = df['Day of Week'].isin([5, 6]).astype(int)

    # Calculate customer metrics
    customer_metrics = calculate_customer_metrics(df)
    df = df.merge(customer_metrics, on='Customer Name')

    return df

def calculate_customer_metrics(df):
    """
    Calculate RFM metrics for each customer
    """
    # Current date for recency calculation
    current_date = df['Order Date'].max()

    customer_metrics = df.groupby('Customer Name').agg({
        'Order Date': lambda x: (current_date - x.max()).days,  # Recency
        'Order ID': 'count',  # Frequency
        'Sales': 'sum'  # Monetary
    }).reset_index()

    customer_metrics.columns = ['Customer Name', 'Recency', 'Frequency', 'Total_Monetary']
    return customer_metrics

# 2. Customer Segmentation
def perform_customer_segmentation(df, n_clusters=4):
    """
    Perform customer segmentation using RFM metrics
    """
    # Select features for clustering
    features = ['Recency', 'Frequency', 'Total_Monetary']

    # Standardize features
    scaler = StandardScaler()
    features_scaled = scaler.fit_transform(df[features])

    # Find optimal number of clusters
    silhouette_scores = []
    K = range(2, 8)

    for k in K:
        kmeans = KMeans(n_clusters=k, random_state=42)
        kmeans.fit(features_scaled)
        score = silhouette_score(features_scaled, kmeans.labels_)
        silhouette_scores.append(score)

    optimal_k = K[np.argmax(silhouette_scores)]

    # Perform final clustering
    kmeans = KMeans(n_clusters=optimal_k, random_state=42)
    df['Customer_Segment'] = kmeans.fit_predict(features_scaled)

    return df, kmeans

# 3. Sales Forecasting
def create_sales_forecast(df, forecast_periods=30):
    """
    Create sales forecast using Prophet
    """
    # Prepare data for Prophet
    daily_sales = df.groupby('Order Date')['Sales'].sum().reset_index()
    daily_sales.columns = ['ds', 'y']

    # Initialize and train Prophet model
    model = Prophet(
        yearly_seasonality=True,
        weekly_seasonality=True,
        daily_seasonality=False,
        seasonality_mode='multiplicative'
    )

    # Add holiday effects
    model.add_country_holidays(country_name='IN')

    # Fit model
    model.fit(daily_sales)

    # Create future dates dataframe
    future_dates = model.make_future_dataframe(periods=forecast_periods)

    # Generate forecast
    forecast = model.predict(future_dates)

    return model, forecast

# 4. Store Placement Analysis
def analyze_store_placement(df):
    """
    Analyze optimal store placement based on various metrics
    """
    store_analysis = df.groupby(['Region', 'City']).agg({
        'Sales': 'sum',
        'Profit': 'sum',
        'Order ID': 'count',
        'Customer Name': 'nunique',
        'Discount': 'mean'
    }).reset_index()

    # Calculate performance metrics
    store_analysis['Sales_per_Customer'] = store_analysis['Sales'] / store_analysis['Customer Name']
    store_analysis['Profit_Margin'] = store_analysis['Profit'] / store_analysis['Sales']

    # Create composite score
    weights = {
        'Sales': 0.3,
        'Profit_Margin': 0.3,
        'Customer Name': 0.2,
        'Sales_per_Customer': 0.2
    }

    # Normalize metrics
    for metric in weights.keys():
        if metric in store_analysis.columns:
            store_analysis[f'{metric}_Normalized'] = (
                store_analysis[metric] - store_analysis[metric].min()
            ) / (store_analysis[metric].max() - store_analysis[metric].min())

    # Calculate final score
    store_analysis['Location_Score'] = sum(
        store_analysis[f'{metric}_Normalized'] * weight
        for metric, weight in weights.items()
    )

    return store_analysis

# 5. Product Recommendation System
def build_product_recommendations(df):
    """
    Build a product recommendation system based on customer purchase patterns
    """
    # Create customer-product purchase matrix
    purchase_matrix = pd.crosstab(
        df['Customer Name'],
        df['Category']
    )

    # Train a model to predict product preferences
    X = purchase_matrix.values
    y = df.groupby('Customer Name')['Sales'].mean().values

    # Split data
    X_train, X_test = train_test_split(X, test_size=0.2, random_state=42)
    y_train, y_test = train_test_split(y, test_size=0.2, random_state=42)

    # Train model
    rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
    rf_model.fit(X_train, y_train)

    return rf_model, purchase_matrix

# 6. Main Analysis Pipeline
def run_retail_analysis(file_path):
    """
    Run the complete retail analysis pipeline
    """
    # 1. Load and preprocess data
    print("Loading and preprocessing data...")
    df = load_and_preprocess_data(file_path)

    # 2. Perform customer segmentation
    print("Performing customer segmentation...")
    df, kmeans_model = perform_customer_segmentation(df)

    # 3. Create sales forecast
    print("Creating sales forecast...")
    prophet_model, forecast = create_sales_forecast(df)

    # 4. Analyze store placement
    print("Analyzing store placement...")
    store_analysis = analyze_store_placement(df)

    # 5. Build product recommendations
    print("Building product recommendations...")
    rec_model, purchase_matrix = build_product_recommendations(df)

    return {
        'processed_data': df,
        'segmentation_model': kmeans_model,
        'forecast_model': prophet_model,
        'forecast_results': forecast,
        'store_analysis': store_analysis,
        'recommendation_model': rec_model,
        'purchase_matrix': purchase_matrix
    }

c:\Users\Rajesh pandey\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def standardize_date_format(df, date_column='Order Date'):
    """
    Convert various date formats to standard 'dd-mm-yyyy' format

    Parameters:
    df (pandas.DataFrame): Input dataframe
    date_column (str): Name of the date column to standardize

    Returns:
    pandas.DataFrame: DataFrame with standardized date format
    """
    def convert_date(date_str):
        try:
            # First try the mm/dd/yyyy format
            if isinstance(date_str, str) and '/' in date_str:
                date_obj = datetime.strptime(date_str, '%m/%d/%Y')
                return date_obj.strftime('%d-%m-%Y')

            # Try dd-mm-yyyy format
            elif isinstance(date_str, str) and '-' in date_str:
                # Verify if it's already in correct format
                datetime.strptime(date_str, '%d-%m-%Y')
                return date_str

            # If date is already a datetime object
            elif isinstance(date_str, datetime):
                return date_str.strftime('%d-%m-%Y')

            else:
                # Try parsing with pandas (handles more formats)
                return pd.to_datetime(date_str).strftime('%d-%m-%Y')

        except Exception as e:
            print(f"Error converting date '{date_str}': {str(e)}")
            return None

    # Create a copy of the dataframe to avoid modifying the original
    df_copy = df.copy()

    # Convert dates
    df_copy[date_column] = df_copy[date_column].apply(convert_date)

    # Remove any rows where date conversion failed
    invalid_dates = df_copy[date_column].isna()
    if invalid_dates.any():
        print(f"Warning: {invalid_dates.sum()} dates could not be converted and will be removed")
        df_copy = df_copy.dropna(subset=[date_column])

    return df_copy

In [3]:
# Load your data
df = pd.read_csv('DMart_Grocery_Sales_-_Retail_Analytics_Dataset.csv')

# Convert dates
df_standardized = standardize_date_format(df, date_column='Order Date')

In [4]:
def run_retail_analysis(df):
    """
    Run the complete retail analysis pipeline

    Parameters:
    df (pandas.DataFrame): Input dataframe with standardized date format
    """
    try:
        # 1. Initial preprocessing
        print("Preprocessing data...")
        processed_df = df.copy()

        # Convert Order Date to datetime if it isn't already
        processed_df['Order Date'] = pd.to_datetime(processed_df['Order Date'], format='%d-%m-%Y')

        # Create time-based features
        processed_df['Year'] = processed_df['Order Date'].dt.year
        processed_df['Month'] = processed_df['Order Date'].dt.month
        processed_df['Day of Week'] = processed_df['Order Date'].dt.dayofweek
        processed_df['Is Weekend'] = processed_df['Day of Week'].isin([5, 6]).astype(int)

        # 2. Calculate customer metrics
        print("Calculating customer metrics...")
        current_date = processed_df['Order Date'].max()

        customer_metrics = processed_df.groupby('Customer Name').agg({
            'Order Date': lambda x: (current_date - x.max()).days,  # Recency
            'Order ID': 'count',  # Frequency
            'Sales': 'sum'  # Monetary
        }).reset_index()

        customer_metrics.columns = ['Customer Name', 'Recency', 'Frequency', 'Total_Monetary']
        processed_df = processed_df.merge(customer_metrics, on='Customer Name')

        # 3. Customer Segmentation
        print("Performing customer segmentation...")
        features = ['Recency', 'Frequency', 'Total_Monetary']
        scaler = StandardScaler()
        features_scaled = scaler.fit_transform(customer_metrics[features])

        kmeans = KMeans(n_clusters=4, random_state=42)
        customer_metrics['Customer_Segment'] = kmeans.fit_predict(features_scaled)
        processed_df = processed_df.merge(
            customer_metrics[['Customer Name', 'Customer_Segment']],
            on='Customer Name'
        )

        # 4. Sales Forecasting
        print("Creating sales forecast...")
        daily_sales = processed_df.groupby('Order Date')['Sales'].sum().reset_index()
        daily_sales.columns = ['ds', 'y']

        prophet_model = Prophet(
            yearly_seasonality=True,
            weekly_seasonality=True,
            daily_seasonality=False
        )
        prophet_model.fit(daily_sales)

        future_dates = prophet_model.make_future_dataframe(periods=30)
        forecast = prophet_model.predict(future_dates)

        # 5. Store Placement Analysis
        print("Analyzing store placement...")
        store_analysis = processed_df.groupby(['Region', 'City']).agg({
            'Sales': 'sum',
            'Profit': 'sum',
            'Order ID': 'count',
            'Customer Name': 'nunique',
            'Discount': 'mean'
        }).reset_index()

        # Calculate performance metrics
        store_analysis['Sales_per_Customer'] = store_analysis['Sales'] / store_analysis['Customer Name']
        store_analysis['Profit_Margin'] = store_analysis['Profit'] / store_analysis['Sales']

        # 6. Product Recommendations
        print("Building product recommendations...")
        purchase_matrix = pd.crosstab(
            processed_df['Customer Name'],
            processed_df['Category']
        )

        X = purchase_matrix.values
        y = processed_df.groupby('Customer Name')['Sales'].mean().values

        rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
        rf_model.fit(X, y)

        return {
            'processed_data': processed_df,
            'segmentation_model': kmeans,
            'forecast_model': prophet_model,
            'forecast_results': forecast,
            'store_analysis': store_analysis,
            'recommendation_model': rf_model,
            'purchase_matrix': purchase_matrix
        }

    except Exception as e:
        print(f"Error in analysis pipeline: {str(e)}")
        raise

# Function to display key insights
def display_insights(results):
    """
    Display key insights from the analysis
    """
    processed_data = results['processed_data']
    forecast = results['forecast_results']
    store_analysis = results['store_analysis']

    print("\n=== ANALYSIS INSIGHTS ===")

    # Customer Segments
    print("\nCustomer Segments:")
    segment_stats = processed_data.groupby('Customer_Segment').agg({
        'Customer Name': 'nunique',
        'Sales': 'mean',
        'Frequency': 'mean'
    }).round(2)
    print(segment_stats)

    # Top Performing Regions
    print("\nTop Performing Regions:")
    top_regions = store_analysis.groupby('Region')['Sales'].sum().sort_values(ascending=False)
    print(top_regions)

    # Sales Forecast
    print("\nSales Forecast (Next 30 days):")
    forecast_summary = forecast.tail(30)[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].describe()
    print(forecast_summary)

In [5]:
# First, standardize your dates
df_standardized = standardize_date_format(df, date_column='Order Date')

# Run the complete analysis
try:
    results = run_retail_analysis(df_standardized)

    # Display insights
    display_insights(results)

    # Access individual components
    processed_data = results['processed_data']
    forecast = results['forecast_results']
    store_analysis = results['store_analysis']

    # Example: Print the first few rows of processed data
    print("\nProcessed Data Sample:")
    print(processed_data.head())

    # Example: Print forecast for next week
    print("\nNext Week's Sales Forecast:")
    print(forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(7))

    # Example: Print top performing stores
    print("\nTop Performing Stores:")
    print(store_analysis.nlargest(5, 'Sales')[['Region', 'City', 'Sales', 'Profit_Margin']])

except Exception as e:
    print(f"Error running analysis: {str(e)}")

Preprocessing data...
Calculating customer metrics...
Performing customer segmentation...
Creating sales forecast...


07:18:55 - cmdstanpy - INFO - Chain [1] start processing
07:18:55 - cmdstanpy - INFO - Chain [1] done processing


Analyzing store placement...
Building product recommendations...

=== ANALYSIS INSIGHTS ===

Customer Segments:
                  Customer Name    Sales  Frequency
Customer_Segment                                   
0                            11  1482.76     201.54
1                            21  1509.05     198.69
2                             9  1501.12     215.89
3                             9  1478.61     185.60

Top Performing Regions:
Region
West       4798743
East       4248368
Central    3468156
South      2440461
North         1254
Name: Sales, dtype: int64

Sales Forecast (Next 30 days):
                        ds          yhat   yhat_lower    yhat_upper
count                   30     30.000000    30.000000     30.000000
mean   2019-01-14 12:00:00  15083.733862  3827.456787  26352.765114
min    2018-12-31 00:00:00  10112.243670 -1015.434634  20562.915038
25%    2019-01-07 06:00:00  13055.558048  1735.068912  25154.891569
50%    2019-01-14 12:00:00  14567.264777  3397.4592

In [13]:
def generate_personalized_recommendations(df, customer_name='Harish'):
    """
    Generate personalized product recommendations for a DMart customer based on their purchase history,
    regional preferences, and sub-category analysis
    
    Parameters:
    df (pandas.DataFrame): DataFrame with columns [Order ID, Customer Name, Category, Sub Category, 
                          City, Order Date, Region, Sales, Discount, Profit, State]
    customer_name (str): Name of the customer to generate recommendations for
    
    Returns:
    dict: Dictionary containing recommendations and supporting data
    """
    # Get customer's purchase history
    customer_purchases = df[df['Customer Name'] == customer_name]
    
    if len(customer_purchases) == 0:
        return {"error": f"Customer {customer_name} not found in the dataset"}
    
    # Get customer's region and city
    customer_region = customer_purchases['Region'].iloc[0]
    customer_city = customer_purchases['City'].iloc[0]
    
    # Calculate customer's category and sub-category preferences
    category_preferences = customer_purchases.groupby(['Category', 'Sub Category']).agg({
        'Order ID': 'count',
        'Sales': 'sum',
        'Profit': 'sum',
        'Discount': 'mean'
    }).reset_index()
    
    category_preferences['Purchase_Frequency'] = category_preferences['Order ID']
    category_preferences['Avg_Spend'] = category_preferences['Sales'] / category_preferences['Order ID']
    category_preferences['Profit_Margin'] = category_preferences['Profit'] / category_preferences['Sales']
    
    # Get regional preferences (from same region)
    regional_preferences = df[df['Region'] == customer_region].groupby(['Category', 'Sub Category']).agg({
        'Order ID': 'count',
        'Sales': 'sum',
        'Customer Name': 'nunique',
        'Profit': 'sum'
    }).reset_index()
    
    regional_preferences['Regional_Popularity'] = regional_preferences['Order ID']
    regional_preferences['Customer_Base'] = regional_preferences['Customer Name']
    regional_preferences['Regional_Profit_Margin'] = regional_preferences['Profit'] / regional_preferences['Sales']
    
    # Merge customer and regional preferences
    recommendations = category_preferences.merge(
        regional_preferences[['Category', 'Sub Category', 'Regional_Popularity', 
                            'Customer_Base', 'Regional_Profit_Margin']], 
        on=['Category', 'Sub Category'], 
        how='outer'
    ).fillna(0)
    
    # Calculate recommendation score
    recommendations['Score'] = (
        0.35 * (recommendations['Purchase_Frequency'] / recommendations['Purchase_Frequency'].max()) +  # Personal preference
        0.25 * (recommendations['Avg_Spend'] / recommendations['Avg_Spend'].max()) +                   # Spending pattern
        0.20 * (recommendations['Regional_Popularity'] / recommendations['Regional_Popularity'].max()) + # Regional popularity
        0.20 * (1 - recommendations['Discount'])                                                        # Price sensitivity
    )
    
    # Get categories the customer hasn't purchased but are popular in their region
    purchased_categories = set(zip(category_preferences['Category'], category_preferences['Sub Category']))
    new_recommendations = regional_preferences[
        ~regional_preferences.set_index(['Category', 'Sub Category']).index.isin(purchased_categories)
    ].sort_values('Regional_Popularity', ascending=False)
    
    # Sort recommendations by score
    recommendations = recommendations.sort_values('Score', ascending=False)
    
    return {
        'top_recommendations': recommendations[['Category', 'Sub Category', 'Score']].head(5),
        'new_categories': new_recommendations[['Category', 'Sub Category', 'Regional_Popularity']].head(3),
        'customer_region': customer_region,
        'customer_city': customer_city,
        'purchase_history': category_preferences[['Category', 'Sub Category', 'Purchase_Frequency', 
                                               'Avg_Spend', 'Discount']],
        'regional_trends': regional_preferences[['Category', 'Sub Category', 'Regional_Popularity', 
                                              'Customer_Base']]
    }

def display_recommendations(recommendations):
    """
    Display the recommendations in a formatted way
    """
    if 'error' in recommendations:
        print(f"Error: {recommendations['error']}")
        return
        
    print(f"\nPersonalized Recommendations for Customer in {recommendations['customer_city']}, {recommendations['customer_region']}")
    print("\n1. Top 5 Recommended Categories Based on Your Shopping Pattern:")
    print("=" * 80)
    for idx, row in recommendations['top_recommendations'].iterrows():
        print(f"{idx + 1}. Category: {row['Category']:<20} Sub-Category: {row['Sub Category']:<20}")
        print(f"   Recommendation Score: {row['Score']:.2f}")
    
    print("\n2. New Categories Popular in Your Region:")
    print("=" * 80)
    for idx, row in recommendations['new_categories'].iterrows():
        print(f"{idx + 1}. Category: {row['Category']:<20} Sub-Category: {row['Sub Category']:<20}")
        print(f"   Regional Orders: {int(row['Regional_Popularity'])}")
    
    print("\n3. Your Shopping Pattern Summary:")
    print("=" * 80)
    history = recommendations['purchase_history']
    for idx, row in history.iterrows():
        print(f"Category: {row['Category']:<20} Sub-Category: {row['Sub Category']:<20}")
        print(f"Orders: {int(row['Purchase_Frequency']):>3}, Average Spend: ₹{row['Avg_Spend']:,.2f}, " \
              f"Typical Discount: {row['Discount']*100:.1f}%")

# Example usage:
results = generate_personalized_recommendations(df, customer_name='Sudha')
display_recommendations(results)


Personalized Recommendations for Customer in Krishnagiri, South

1. Top 5 Recommended Categories Based on Your Shopping Pattern:
1. Category: Bakery               Sub-Category: Biscuits            
   Recommendation Score: 0.83
4. Category: Beverages            Sub-Category: Health Drinks       
   Recommendation Score: 0.82
5. Category: Beverages            Sub-Category: Soft Drinks         
   Recommendation Score: 0.79
9. Category: Eggs, Meat & Fish    Sub-Category: Mutton              
   Recommendation Score: 0.75
18. Category: Oil & Masala         Sub-Category: Edible Oil & Ghee   
   Recommendation Score: 0.75

2. New Categories Popular in Your Region:

3. Your Shopping Pattern Summary:
Category: Bakery               Sub-Category: Biscuits            
Orders:  13, Average Spend: ₹1,476.15, Typical Discount: 26.7%
Category: Bakery               Sub-Category: Breads & Buns       
Orders:   9, Average Spend: ₹1,546.67, Typical Discount: 24.9%
Category: Bakery               Sub-Cat